In [4]:
import requests
from bs4 import BeautifulSoup

# URL de la page web
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Envoyer une requête GET pour récupérer le contenu HTML de la page
response = requests.get(url)

# Vérifier que la requête a réussi
if response.status_code == 200:
    # Parser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Trouver la div avec une classe spécifique (par exemple "example-class")
    div_content = soup.find("tbody", class_="")
    
    # Vérifier si la div existe et récupérer son contenu
    if div_content:
        print("Contenu de la div :")
        print(div_content.text.strip())  # Récupère uniquement le texte à l'intérieur de la div
    else:
        print("La div avec cette classe n'existe pas sur la page.")
else:
    print(f"Échec de la requête : {response.status_code}")


Contenu de la div :
January

Yellow Taxi Trip Records (PARQUET)
Green Taxi Trip Records (PARQUET)
For-Hire Vehicle Trip Records (PARQUET)
High Volume For-Hire Vehicle Trip Records (PARQUET)

February

Yellow Taxi Trip Records (PARQUET)
Green Taxi Trip Records (PARQUET)
For-Hire Vehicle Trip Records (PARQUET)
High Volume For-Hire Vehicle Trip Records (PARQUET)

March

Yellow Taxi Trip Records (PARQUET)
Green Taxi Trip Records (PARQUET)
For-Hire Vehicle Trip Records (PARQUET)
High Volume For-Hire Vehicle Trip Records (PARQUET)

April

Yellow Taxi Trip Records (PARQUET)
Green Taxi Trip Records (PARQUET)
For-Hire Vehicle Trip Records (PARQUET)
High Volume For-Hire Vehicle Trip Records (PARQUET)

May

Yellow Taxi Trip Records (PARQUET)
Green Taxi Trip Records (PARQUET)
For-Hire Vehicle Trip Records (PARQUET)
High Volume For-Hire Vehicle Trip Records (PARQUET)

June

Yellow Taxi Trip Records (PARQUET)
Green Taxi Trip Records (PARQUET)
For-Hire Vehicle Trip Records (PARQUET)
High Volume For-H

In [5]:

# URL de la page web
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Envoyer une requête GET pour récupérer le contenu HTML de la page
response = requests.get(url)

# Parse le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Chercher le lien pour "Yellow Taxi Trip Records" de janvier
yellow_taxi_january = soup.find("a", title="Yellow Taxi Trip Records")

# Récupérer l'URL du fichier Parquet
if yellow_taxi_january:
    link = yellow_taxi_january["href"]
    print("Lien du fichier Parquet pour Yellow Taxi Trip Records (January 2024) :")
    print(link)
else:
    print("Le lien pour Yellow Taxi Trip Records de January 2024 n'a pas été trouvé.")
    

Lien du fichier Parquet pour Yellow Taxi Trip Records (January 2024) :
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet 


In [12]:
import os

# Télécharger le fichier Parquet
file_name = os.path.join("../data", link.split("/")[-1])  # Nom du fichier basé sur le lien
print(f"Téléchargement du fichier dans : {file_name}")
print(link)
    
# Ajouter un User-Agent qui imite un navigateur
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(link, headers=headers, stream=True)

if response.status_code == 200:
        with open(file_name, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):  # Télécharger par morceaux
                f.write(chunk)
        print("Téléchargement terminé avec succès.")
else:
        print(f"Erreur lors du téléchargement. Code HTTP : {response.status_code}")
        print(response)


Téléchargement du fichier dans : ../data/yellow_tripdata_2024-01.parquet 
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet 
Erreur lors du téléchargement. Code HTTP : 403
<Response [403]>


In [13]:
%pip install selenium

     |████████████████████████████████| 9.7 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 8.9 MB/s  eta 0:00:01
     |████████████████████████████████| 481 kB 40.6 MB/s eta 0:00:01
  Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached exceptiongroup-1.2.2-py3-none-any.whl (16 kB)
     |████████████████████████████████| 58 kB 6.6 MB/s eta 0:00:011
You should consider upgrading via the '/Users/belhadri-youcef/Documents/Personnel/Formation/data-projects/NYC-Taxi/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os
import time
import pandas as pd

# Configurer les options Chrome
options = Options()
options.add_argument("--headless")  # Mode headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_experimental_option("prefs", {
    "download.default_directory": os.path.abspath("data"),  # Dossier de téléchargement
    "download.prompt_for_download": False,
})

# Service ChromeDriver
service = Service("/usr/local/bin/chromedriver")  # Chemin vers ChromeDriver

# Initialiser le navigateur
driver = webdriver.Chrome(service=service, options=options)

try:
    # URL du fichier à télécharger
    url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
    driver.get(url)

    # Pause pour laisser le téléchargement se faire
    time.sleep(10)
    print("Téléchargement terminé")
    # Vérifier la taille du fichier téléchargé
    download_folder = os.path.abspath("data")
    file_name = "yellow_tripdata_2024-01.parquet"  # Nom du fichier téléchargé
    file_path = os.path.join(download_folder, file_name)

    if os.path.exists(file_path):
        # Taille du fichier en octets
        file_size_bytes = os.path.getsize(file_path)
        
        # Convertir en Mo
        file_size_mb = file_size_bytes / (1024 * 1024)  # Mo
        print(f"Taille du fichier téléchargé : {file_size_mb:.2f} Mo")
        
        # Charger le fichier dans un DataFrame pandas
        df = pd.read_parquet(file_path)
        print("Le fichier a été chargé dans le DataFrame.")
        print(df.head(3))  # Afficher les premières lignes du DataFrame
        
    else:
        print(f"Le fichier {file_name} n'a pas été trouvé dans le dossier de téléchargement.")
finally:
    driver.quit()



Téléchargement terminé
Taille du fichier téléchargé : 47.65 Mo
Le fichier a été chargé dans le DataFrame.
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2024-01-01 00:57:55   2024-01-01 01:17:43              1.0   
1         1  2024-01-01 00:03:00   2024-01-01 00:09:36              1.0   
2         1  2024-01-01 00:17:06   2024-01-01 00:35:01              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.72         1.0                  N           186            79   
1           1.80         1.0                  N           140           236   
2           4.70         1.0                  N           236            79   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \
0             2         17.7    1.0      0.5        0.00           0.0   
1             1         10.0    3.5      0.5        3.75           0.0   
2             1         23.3    3.5      0.5        3.